# Wasserstein GAN (WGAN) by `Mr. Harshit Dawar!`
* Addresses the following problems of DCGANs
    * Mode Collapse
    * Vanishing Gradient
* It also requires less epochs to provide better results as compared to the DCGANs
* It takes more time to train as compared to DCGANs